In [1]:
#Representacion de los estados para la resolaucion del problema de los ascensores. 
#aaa

#Estado Inicial: 
#          * personas indicando la plata actual en la que estan
#ini = ((1, 3, 4), ((0,[]),(0,[])))
ini = (((0,2), (0,4), (0,1),(1,8),(0,1)), ((0,0), (1,5), (1,6), (2, 10)))
#Reresentacion des estado final solo de las personas ya que los ascensores no importa doende 
fin = ((0,3), (2,11), (2,12),(0,1), (1,9)) 
####ini = (((0,0), (0,4)),((0,0),(2,12)))
####fin = ((0,1),(0,0))
#                   * Ascensores tambien indicando en la planta actual en la que estan. 


#acaben 


In [2]:
# importamos las cosas que vamos a usar de aima

        
        

from search import *

class Ascensores(Problem):
    ''' Clase Ascensores la cual implenta la clase abstracta Problema que nos ayuda a resover el porblema
           con varias funciones de busqueda '''
    def __init__(self,initial, goal):
        '''Metodo que inicializa nustro problema'''
        
        Problem.__init__(self, initial, goal)
        #self.search = initial
        self.goal = goal
        
        

    def actions(self, estado):
        '''Apartir de un estado(n) dado nos lista un conjunto de acciones(a) que le podimos aplicar a n para
        alcanzar otro estado n'  ''' 
        
        listaPersonas = list(estado[0])
        listAscensores = list(estado[1])
        
        acciones = list() 
      
        for i in range (len(listaPersonas)):
            persona = listaPersonas[i] #tupla (bloque, pisoAct)
            
            for j in range (len(listAscensores)):
                ascensor = listAscensores[j] #tupla (bloque, pisoAct)
                if ((ascensor[0] == 0 and ascensor[1] < 4) or (ascensor[0] == 1 and ascensor[1] < 8) or (ascensor[0] == 2 and ascensor[1] < 12)):
                    acciones.append((i,j, "Subir ascensor")) # sube al ascensor j a la planta donde esta la persona i
                if  ((ascensor[0] == 0 and ascensor[1] > 0) or (ascensor[0] == 1 and ascensor[1] > 4) or (ascensor[0] == 2 and ascensor[1] > 8)):
                    acciones.append((i,j, "Bajar ascensor"))
                if persona == ascensor and persona[1] < 12: # para si la persona esta en la ultima planta solo
                    acciones.append((i,j, "Bajar pasajero"))           #pordria bajar
                if persona == ascensor and persona[1] > 0: # Idem pero al reves 
                    acciones.append((i,j, "Subir pasajero"))
                
                #en el momento que ascensor este en la misma planta que la persona el arbol se ramifica en dos caminos
                #por lo que se generaran estas dos acciones. Quien decida que accion utilizar será la heuristica 
                #puesto que cogere la que menos heuristica tenga ya que estará mas cerca de la solución
                
        return acciones; 

        
    #accion = (plantaPersona,PlantaAscensor, accion en string)
    def result(self, estado, accion):
        '''Dado un estado(n, nodo actual) y una accion(a) nos permite general un nuevo nodo(n') aplicando la accion dada'''
        listaPersonas = list(estado[0]) #la primera(0)posicion de la tupla de estado la convertimos a una lista
        listAscensores = list(estado[1])# Idema para los ascensores con la segunda posicion (1)

        persona = accion[0] # Extraemos la posicion de la listaen donde se encutra la persona a modificar su posicion en el edificio
        ascensor = accion[1]# Idem con los ascensores 
                #Se hace simplemente para facilitar la lectura del codigo

        actualPersona = list(listaPersonas[persona]) #Planta actual donde se encutra la persona (bloque, posAct)
        actualAscensor = list(listAscensores[ascensor])#idem con el ascensor (bloque, posAct)

        if accion[2] == "Subir ascensor":
            actualAscensor[1] += 1 # Si es llamar a la posicion actual le asignamos la posicion 
                    #actual de la persona que ha solicitado el ascensor
            listaPersonas[persona] = tuple(actualPersona)
            listAscensores[ascensor] = tuple(actualAscensor)
        elif accion[2] == "Bajar ascensor":
            actualAscensor[1] -= 1 # Si es llamar a la posicion actual le asignamos la posicion 
                    #actual de la persona que ha solicitado el ascensor
            listaPersonas[persona] = tuple(actualPersona)
            listAscensores[ascensor] = tuple(actualAscensor)
        elif accion[2] == "Bajar pasajero": #Si es bajar restamos en uno la posicion actual de ambos(bajan un piso)
            actualPersona[1] -= 1 
            actualAscensor[1] -=1
            listaPersonas[persona] = tuple(actualPersona)
            listAscensores[ascensor] = tuple(actualAscensor)
            if(actualPersona[1]<4):
                actualPersona[0] = 0
            elif(actualPersona[0] < 8):
                actualPersona[0] = 1
            elif(actualPersona[0] <= 12):
                actualPersona[0] = 2


        elif accion[2] == "Subir pasajero": #Si es Subir sumamos en uno la posicion actual de ambos(sube un piso)
            actualPersona[1] += 1
            actualAscensor[1] +=1
            listaPersonas[persona] = tuple(actualPersona)
            listAscensores[ascensor] = tuple(actualAscensor)
            if(actualPersona[1]<4):
                actualPersona[0] = 0
            elif(actualPersona[0] < 8):
                actualPersona[0] = 1
            elif(actualPersona[0] < 12):
                actualPersona[0] = 2
        
        return (tuple(listaPersonas), tuple(listAscensores))

    def goal_test(self, estado): # Devuelve true si se ha alcanzado el estado objetivo 
        return estado[0] == self.goal 


In [3]:
estado = ini
accion = (0,0, "Mover ascensor")

l = list(estado)
listaPersonas = list(estado[0]) #la primera(0)posicion de la tupla de estado la convertimos a una lista
listAscensores = list(estado[1])# Idema para los ascensores con la segunda posicion (1)
        
persona = accion[0] # Extraemos la posicion de la listaen donde se encutra la persona a modificar su posicion en el edificio
ascensor = accion[1]# Idem con los ascensores 
        #Se hace simplemente para facilitar la lectura del codigo
        
actualPersona = list(listaPersonas[persona]) #Planta actual donde se encutra la persona (bloque, posAct)
actualAscensor = list(listAscensores[ascensor])#idem con el ascensor (bloque, posAct)
        
if accion[2] == "Mover ascensor":
    actualAscensor[1] = int(actualPersona[1]) # Si es llamar a la posicion actual le asignamos la posicion 
            #actual de la persona que ha solicitado el ascensor
    listaPersonas[persona] = tuple(actualPersona)
    listAscensores[ascensor] = tuple(actualAscensor)
if accion[2] == "Bajar pasajero": #Si es bajar restamos en uno la posicion actual de ambos(bajan un piso)
    actualPersona[1] -= 1 
    actualAscensor[1] -=1
    listaPersonas[persona] = tuple(actualPersona)
    listAscensores[ascensor] = tuple(actualAscensor)
if accion[2] == "Subir pasajero": #Si es Subir sumamos en uno la posicion actual de ambos(sube un piso)
    actualPersona[1] += 1
    actualAscensor[1] +=1
    listaPersonas[persona] = tuple(actualPersona)
    listAscensores[ascensor] = tuple(actualAscensor)
        
print(tuple(listaPersonas), tuple(listAscensores))

((0, 2), (0, 4), (0, 1), (1, 8), (0, 1)) ((0, 2), (1, 5), (1, 6), (2, 10))


In [4]:
p = Ascensores(ini, fin)
p.initial

(((0, 2), (0, 4), (0, 1), (1, 8), (0, 1)), ((0, 0), (1, 5), (1, 6), (2, 10)))

In [5]:
p.result((((0, 1), (1, 7), (2, 11)), ((0, 0), (1, 5), (1, 6), (2, 10))), (0, 0, "Mover ascensor"))

(((0, 1), (1, 7), (2, 11)), ((0, 0), (1, 5), (1, 6), (2, 10)))

In [6]:
p.actions(p.initial)

[(0, 0, 'Subir ascensor'),
 (0, 1, 'Subir ascensor'),
 (0, 1, 'Bajar ascensor'),
 (0, 2, 'Subir ascensor'),
 (0, 2, 'Bajar ascensor'),
 (0, 3, 'Subir ascensor'),
 (0, 3, 'Bajar ascensor'),
 (1, 0, 'Subir ascensor'),
 (1, 1, 'Subir ascensor'),
 (1, 1, 'Bajar ascensor'),
 (1, 2, 'Subir ascensor'),
 (1, 2, 'Bajar ascensor'),
 (1, 3, 'Subir ascensor'),
 (1, 3, 'Bajar ascensor'),
 (2, 0, 'Subir ascensor'),
 (2, 1, 'Subir ascensor'),
 (2, 1, 'Bajar ascensor'),
 (2, 2, 'Subir ascensor'),
 (2, 2, 'Bajar ascensor'),
 (2, 3, 'Subir ascensor'),
 (2, 3, 'Bajar ascensor'),
 (3, 0, 'Subir ascensor'),
 (3, 1, 'Subir ascensor'),
 (3, 1, 'Bajar ascensor'),
 (3, 2, 'Subir ascensor'),
 (3, 2, 'Bajar ascensor'),
 (3, 3, 'Subir ascensor'),
 (3, 3, 'Bajar ascensor'),
 (4, 0, 'Subir ascensor'),
 (4, 1, 'Subir ascensor'),
 (4, 1, 'Bajar ascensor'),
 (4, 2, 'Subir ascensor'),
 (4, 2, 'Bajar ascensor'),
 (4, 3, 'Subir ascensor'),
 (4, 3, 'Bajar ascensor')]

In [7]:
p.result(p.initial, (0,0, "Mover ascensor"))

(((0, 2), (0, 4), (0, 1), (1, 8), (0, 1)), ((0, 0), (1, 5), (1, 6), (2, 10)))

In [8]:
# Importamos los modulos pertenecinetes a los algoritmos de busqueda 
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [9]:
#%%time
#breadth_first_graph_search(p).solution()

In [10]:
#Definimos algunas Hehuristicas 

def linear(node): # dado un estado nos devuelve las personas que todavia no estan en su planta objetivo
# es decir, que estan mal colocadas (como en el puzzle de 8)
    result = 0
    goal = fin
    estado = node.state[0]
    
    for i in range (len(estado)):
        if goal[i] != estado[i]:
            result += 1
    return result

# Sería como el de distancia hasta su objetivo quitando los if que comprueban las paradas para cambio de ascensor
def paradasTotales(node):
    result = 0
    bloque1 = (0,1,2,3,4)
    bloque2 = (4,5,6,7,8)
    bloque3 = (8,9,10,11,12)
    
    goal = fin
    estado = node.state[0] 
    
    for i in range (len(estado)):
        plantaIni = estado[i][1]
        plantaFin = goal[i][1]
        #podemos ignorar la comporacion de goal[i] == estado[i] ya que el resultado es 0
        result += abs(plantaFin - plantaIni)
        
        if plantaFin != plantaIni: 
        #Hago esto con conjuntos para determinar las paradas que han de hacer los ascensores Lentos
        #Por ejemplo si el inicio esta en el bloque 1 y el objetivo es el bloque3 entonces ha de hacer dos paradas
            if plantaIni < plantaFin: #Sube 
                if plantaFin in bloque2 and plantaIni in bloque1: 
                    result += 1
                elif plantaFin in bloque3 and plantaIni in bloque1:
                    result +=2
                elif plantaFin in bloque3 and plantaIni in bloque2:
                    result += 1
            else: # Baja
                if plantaIni in bloque3 and plantaFin in bloque2: 
                    result += 1
                elif plantaIni in bloque3 and plantaFin in bloque1:
                    result +=2
                elif plantaIni in bloque2 and plantaFin in bloque1:
                    result += 1
                
    return result

def distanciaParaObjetivo(node):
    result = 0
    goal = fin
    estado = node.state[0]
    
    for i in range (len(estado)):
        #print(i)
        if(goal[i][1] != estado[i][1]):
            result += abs(goal[i][1] - estado[i][1])
    
    return result
        
    
    


In [ ]:
%%time
astar_search(p, distanciaParaObjetivo).solution()